In [1]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# FROM BELOW ARE NOTES FOR MAIN FILE V2 "gpt.py"

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

In [3]:
torch.manual_seed(1337)

In [8]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2024-07-28 08:36:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2024-07-28 08:36:25 (175 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [9]:
# here are all the unique characters that occur in this text (ALREADY NOTED, REFER TO PREVIOUS V1 FILE FOR WORKINGS)
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [10]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generates tensor of random indices, shape tuple defined by batch_size // These indices serves to index the data file. See 2 lines below.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [12]:
@torch.no_grad() # disables gradient calculation (used during evaluation)
def estimate_loss():
    out = {} # empty dict which function will store the average losses for each data split, "train" and "val"

    model.eval() # this method switches the model to evaluation model. Which affects layers like dropout layers that have different behaviours during training vs evaluation (dropout is turned off during evaluation)
    for split in ['train', 'val']: # begins a loop over the data splits. The function evaluates the model both on training data and validation data. Allowing you o monitor overfitting and general performance.
        losses = torch.zeros(eval_iters) # initialises tensor "losses" filled with zeroes of length eval_iters
        for k in range(eval_iters):
            X, Y = get_batch(split) # uses get_batch with current split to generate a bunch of input data X and target data Y
            logits, loss = model(X, Y) # passes the batch through the model. The model returns logits (raw outputs before applying softmax), and the computed "loss" for this batch.
            losses[k] = loss.item()  # extracts value of loss using ".item()" and stores it in the losses tensor at index "k"
        out[split] = losses.mean() # after all iterations for current split is complete. Calculates the mean of all recorded losses. And stores it in dictionary "out", where split is the key.
    model.train() # switches model back to training mode after evaluating on both splits. Re-enabling training specific behaviours like dropout
    return out

## Notes for class Head

You're on the right track in understanding the `nn.Linear` layer in PyTorch. Here's a detailed explanation of what it does and how it's configured in your code:

### nn.Linear Function
`nn.Linear` is a PyTorch module that applies a linear transformation to the incoming data. It's essentially a fully connected neural network layer. Here’s what each parameter in the `nn.Linear` initialization means:

- **n_embd (input features):** This is the size of each input sample. For instance, if `n_embd` is 512, each input to the layer should have 512 features.

- **head_size (output features):** This is the size of each output sample. The layer transforms the input dimension (`n_embd`) into the `head_size` dimension. For example, if `head_size` is 64, each output from this layer will have 64 features.

- **bias (Boolean):** This is a flag that indicates whether a bias vector should be added to the output. If `bias=False`, no bias is added. If `bias=True`, a bias vector (initialized to zero by default) is created and added to the outputs.

### Role in the Attention Head
In the context of your `Head` class within a transformer, these layers (`self.key`, `self.query`, `self.value`) are used to transform the input into three different representations:
- **Keys (k):** Used to interact with queries to compute attention scores.
- **Queries (q):** Used to interact with keys to fetch the most relevant information across the sequence.
- **Values (v):** Once the relevant positions are identified using keys and queries, the values at these positions are combined to produce the output.

### Working Mechanism
Here's what happens when you use `nn.Linear` in the context of your self-attention head:

1. **Input Dimensionality:** The input `x` to your `forward` function in the `Head` class has the shape `[B, T, C]`, where `B` is the batch size, `T` is the sequence length (number of time steps), and `C` is the number of channels (here, `C` is `n_embd`, the embedding size).

2. **Transformation:**
   - When `x` is passed through `self.key(x)`, the layer transforms each `[B, T, n_embd]` input into `[B, T, head_size]`. It does this by multiplying `x` by a weight matrix `W` of shape `[n_embd, head_size]`, and since `bias=False`, no bias is added. The same transformation is applied by `self.query(x)` and `self.value(x)` to produce queries and values, respectively.

3. **Output Dimensionality:** Each of the transformed outputs (`k`, `q`, `v`) now has the shape `[B, T, head_size]`, aligning with the required dimensions to compute attention scores and ultimately, the weighted sum of values.

This transformation allows each head to project the input embeddings into a different subspace, helping the model to focus on different features of the input at different positions, enhancing its ability to capture complex relationships in the data.

## Back to Coding

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False) # transformers tensor of [B, T, n_embed] to [B, T, head_size] // after applying a trained linear transformation
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # A lower triangular matrix tril of size (block_size, block_size) is created and registered as
                                                                                     # a buffer. This matrix is used later to apply a mask for the attention mechanism, allowing the model to
                                                                                     # only attend to previous positions and prevent "looking ahead."
        self.dropout = nn.Dropout(dropout) # dropout layer is included to prevent overfitting during training by randomly zeroing out elements of the output tensor with a probability defined by dropout.

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
                                                         # this is the attention(Q, K, V) equation. division by sqrt(d_k) can be seen.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

## Notes of MultiHeadAttention and nn.Module

Certainly! The `MultiHeadAttention` class is a crucial component of Transformer models, extending the concept of self-attention by incorporating multiple heads. Each head can potentially learn to pay attention to different parts of the input, making the model more powerful and versatile. Here’s a detailed breakdown of this class:

### Class Definition
```python
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
```
- The class `MultiHeadAttention` inherits from `nn.Module`, which is a base class for all neural network modules in PyTorch. It manages multiple heads of attention that process the input in parallel, allowing the model to capture different aspects of information simultaneously.

### Constructor
```python
def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # creates x number of parallel self attention heads
    self.proj = nn.Linear(head_size * num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)
```
- **ModuleList of Heads:**
  - `self.heads` is a `nn.ModuleList` containing several instances of the `Head` class defined earlier. The list comprehension `[Head(head_size) for _ in range(num_heads)]` creates `num_heads` instances of `Head`, each capable of transforming the input independently.
  - `head_size` defines the size of each head's output.

- **Projection Layer:**
  - `self.proj` is a linear layer that projects the concatenated outputs of all attention heads back to the original embedding dimension (`n_embd`). This is necessary because each head outputs `head_size` features, and concatenating `num_heads` of them results in `head_size * num_heads` features.
  - This layer maps the combined features back to the expected size for compatibility with other components in the Transformer architecture.

- **Dropout:**
  - `self.dropout` is a dropout layer that randomly zeroes elements of the output tensor with a probability defined by `dropout`. This regularization technique helps prevent overfitting.

### Forward Pass
```python
def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out)) # projection back into the residual pathway
    return out
```
- **Concatenating Heads Outputs:**
  - The forward pass starts by applying each head in `self.heads` to the input `x`. The list comprehension `[h(x) for h in self.heads]` computes the outputs from all heads.
  - `torch.cat([...], dim=-1)` concatenates these outputs along the last dimension. If each head's output has dimensions `[B, T, head_size]` and there are `num_heads` heads, the result will have dimensions `[B, T, head_size * num_heads]`.

- **Projection and Dropout:**
  - The concatenated output is then passed through the `self.proj` linear transformation, which reduces its dimensionality from `head_size * num_heads` back to `n_embd`, aligning it with the rest of the network.
  - After the projection, dropout is applied for regularization.

The `MultiHeadAttention` module effectively combines information from multiple representational spaces. By doing so, it allows the model to attend to information from different subsequences in different positions, which is a powerful mechanism in many NLP tasks. This architecture is one of the key reasons why Transformers excel in handling complex dependencies in sequence data.

`nn.Module` is one of the core building blocks in PyTorch, a popular deep learning framework. It serves as the base class for all neural network modules, and most of the functionality of neural networks in PyTorch is built on top of this class. Here’s a detailed breakdown of what `nn.Module` is, what it contains, and what it does:

### Definition of nn.Module
- **Base Class for All Networks:** `nn.Module` is the base class for all neural network modules in PyTorch. Any new neural network component should inherit from `nn.Module` to get all its functionality.

### Key Features and Functions
1. **Parameter Management:**
   - **Automatic Parameter Registration:** When you define instance attributes that are `nn.Parameter` or `nn.Module` types, they are automatically added to the list of parameters (or sub-modules) that the module knows about. This includes weights, biases, and other parameters which are used in forward passes and are necessary for backpropagation.
   - **Easy Access to Parameters:** `nn.Module` provides methods like `.parameters()` and `.named_parameters()` to iterate over all parameters of the model, which is very useful for optimization, saving, loading, etc.

2. **Sub-modules Management:**
   - **Hierarchical Structure:** You can nest `nn.Module` instances inside one another. This hierarchical organization allows building complex architectures easily. Methods like `.children()` and `.modules()` help in accessing these sub-modules at different levels of hierarchy.

3. **Forward Pass Definition:**
   - **Forward Method:** Each `nn.Module` subclass typically implements a `forward()` method. When you call the module (like a function call with `module(input)`), it internally calls `forward()` with the input. This method is where you define the computation performed by the module.

4. **Gradient Computation and Backpropagation:**
   - **Support for Autograd:** `nn.Module` seamlessly integrates with PyTorch’s autograd system. Parameters of the module are automatically registered for gradient computation. When used in a training loop, gradients are computed when calling `.backward()`, and PyTorch takes care of all the gradient flow calculations through modules.

5. **Utilities for Training:**
   - **to(device):** You can move all module parameters to a specified device (CPU or GPU) with a single call to `.to(device)`.
   - **train() and eval():** Switch between training and evaluation modes. This affects the behavior of certain layers like dropout (active during training and inactive during evaluation) and batch normalization (uses running statistics during evaluation).

6. **Serialization and Deserialization:**
   - **Save and Load Models:** `nn.Module` provides convenient methods for saving (`torch.save(module.state_dict(), PATH)`) and loading (`module.load_state_dict(torch.load(PATH))`) the parameters of a model, allowing for model persistence and transfer.

### Practical Example
When you define a new class that inherits from `nn.Module`, you typically:
1. Initialize the parent class in your constructor.
2. Define any layers or parameters your module needs.
3. Implement the `forward()` method to specify how the module processes input.

Here is a simple example:
```python
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 5)  # A simple linear layer

    def forward(self, x):
        return self.linear(x)
```
In this example, `SimpleModel` contains a single linear layer, and the `forward` method defines how the model processes input `x` through that layer.

### Summary
`nn.Module` is essentially what makes building, training, and using neural networks in PyTorch straightforward and flexible. It provides the infrastructure for assembling layers and parameters into a complete model, managing their states, and using them efficiently during training or inference.

## Back to Coding

In [14]:
class MultiHeadAttention(nn.Module): # inherits from nn.Module <-- base class for all NN modules in PyTorch
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size): # class constructor
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # creates x number of parallel self attention heads
                                                                                # contains several instances of the "Head" class defined earlier. Each capable of transforming the input independently.
        self.proj = nn.Linear(head_size * num_heads, n_embd) # projects the concatenated outputs of all attention heads back to the original embedding dimension "n_embeds". This is
                                                             # necessary because each head outputs head_size features, and concatenating num_heads of them results in head_size * num_heads features.
        self.dropout = nn.Dropout(dropout) # dropout layer

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concatenates the outputs from independent heads along the last dimension. // [B, T, head_size] becomes [B, T, head_size * num_heads]
        out = self.dropout(self.proj(out)) # passes through this linear transformation. Which reduces its dimensionality from head_size*num_heads for dim=-1 to n_embed // after projection, dropout is applied for regularization
        return out

## Notes for FeedForward

The code you've shared defines a class named `FeedForward`, which is a component commonly used in Transformer architectures, especially as specified in the influential paper "Attention is All You Need". This class is designed to process the output of the attention mechanism through a two-layer neural network with a ReLU non-linearity and dropout regularization. Here's a breakdown of the class and its functionalities:

### Class Definition
```python
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
```
- **Class Inheritance:** `FeedForward` inherits from `nn.Module`, making it a module in the PyTorch framework.
- **Documentation String:** The docstring indicates that this module involves a simple linear transformation followed by a non-linear activation function.

### Constructor
```python
def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd), # both this and 2 lines below had been multiplied by 4 based on FFN implementation in the paper "Attention is All You Need"
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd), # this is the projection layer going back into the residual pathway
        nn.Dropout(dropout),
    )
```
- **Initialization:**
  - `super().__init__()` is called to initialize the base class (`nn.Module`).
  - `n_embd` is the embedding dimension (size of each input and output vector of the FeedForward module).

- **Network Composition:** `self.net` is defined as a `nn.Sequential` container which automatically creates a network where the output of one layer is passed as the input to the next.
  - **First Linear Layer:** Expands the dimensionality of the input from `n_embd` to `4 * n_embd`. This expansion is typical in Transformer models as it allows the network to create a richer representation of the data.
  - **ReLU Activation:** Introduces non-linearity into the network, which helps the model learn complex patterns.
  - **Second Linear Layer:** Projects the dimensionality back from `4 * n_embd` to `n_embd`, making the output dimension match the input dimension, which is crucial for the addition of the residual connection in the Transformer block.
  - **Dropout:** Randomly zeroes some of the elements of the output tensor with a probability equal to `dropout`, which is a regularization method to prevent overfitting.

### Forward Pass
```python
def forward(self, x):
    return self.net(x)
```
- **Input and Output:** The forward method simply passes the input `x` through the `nn.Sequential` network defined in the constructor. It outputs the processed tensor which has the same dimension as the input tensor.

### Summary
The `FeedForward` module in the Transformer architecture typically follows the multi-head attention component in each layer of the model. It provides additional processing power by transforming the representations learned by the attention mechanism. The inclusion of non-linearity and expansion in dimensionality allows the network to explore more complex transformations before projecting back to the original dimension, which facilitates learning deeper representations in the context of sequences.

This module's setup, especially with the expansion and contraction of dimensions and the use of dropout and ReLU, is critical for effective learning in deep Transformer networks.

## back to coding

In [15]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # both this and 2 lines below had been multipied by 4 based on FFN implementation in the
                                           # paper "Attention is All you need" // transforms dim=-1 to 4*n_embed to have a richer representation before passing it back to ReLu
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # this is the projection layer going back into the residual pathway
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### Forward Method
```python
def forward(self, x):
    x = x + self.sa(self.ln1(x)) # residual
    x = x + self.ffwd(self.ln2(x)) # from feedforward
    return x
```
- **Processing Flow:**
  - The input `x` first passes through the first layer normalization (`self.ln1`). Normalizing the input to the multi-head attention helps in stabilizing the learning.
  - The output from `self.ln1(x)` is then passed to the multi-head attention module (`self.sa`). The result is added back to the original input `x` in a residual connection. This is a key feature of the Transformer architecture, promoting easier gradient flow through deep networks and allowing the model to effectively learn identity functions, which can be critical in deep networks.
  - The result of the attention and residual addition then passes through the second layer normalization (`self.ln2`). Similar to the first normalization, this helps in preparing the normalized data for subsequent operations.
  - The output from `self.ln2(x)` is fed into the feed-forward network (`self.ffwd`). The output from the feed-forward network is added back to the input of the feed-forward network (another residual connection).

In [16]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head # size of each attention head's output. Calculated by dividing embedding dimension by num_heads // ensures total dimensionality is preserved, when outputs of heads are concatenated
        self.sa = MultiHeadAttention(n_head, head_size) # instance of multi-head attention class
        self.ffwd = FeedFoward(n_embd) # notice how this occurs after attention // feed-forward network component of the trasnformer block.
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd) # 2 instances of nn.LayerNorm for applying layer normalization. // crucial for stabilising training of DNN. Applied before and after the attention and ffwd ops.

    def forward(self, x): # REFER TO NOTES ABOVE FOR PROCESS
        x = x + self.sa(self.ln1(x)) # residual
        x = x + self.ffwd(self.ln2(x)) # from feedforward
        return x

## Notes

The `GPTLanguageModel` class implements a language model based on the GPT (Generative Pre-trained Transformer) architecture using PyTorch. Here's a breakdown of the different components and functionalities within this class:

### Class Definition

```python
class GPTLanguageModel(nn.Module):
```
- **Inheritance from nn.Module**: This class inherits from PyTorch’s `nn.Module`, which is the base class for all neural network modules.

### Constructor

```python
def __init__(self):
    super().__init__()
```
- **Initialization of Superclass**: Initializes the superclass `nn.Module`, enabling the functionality of the PyTorch module system.

```python
self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
self.position_embedding_table = nn.Embedding(block_size, n_embd)
```
- **Embedding Tables**: Initializes two embedding layers.
  - `token_embedding_table`: Maps each token in the vocabulary to a high-dimensional space (`n_embd`).
  - `position_embedding_table`: Maps each position in a sequence (up to `block_size`) to the same high-dimensional space, allowing the model to consider the order of tokens.

```python
self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
```
- **Blocks of Transformer**: Initializes a sequence of transformer blocks (`Block`), which handle the complex interactions between different parts of the input through attention mechanisms and feed-forward networks.

```python
self.ln_f = nn.LayerNorm(n_embd)
self.lm_head = nn.Linear(n_embd, vocab_size)
```
- **Final Layer Normalization and Output Linear Layer**:
  - `ln_f`: Applies normalization to the output of the last transformer block.
  - `lm_head`: Transforms the output from the dimensionality of embeddings (`n_embd`) back to the vocabulary size, producing the logits for each token in the vocabulary.

### Weight Initialization

```python
self.apply(self._init_weights)
```
- **Custom Weight Initialization**: Applies a custom weight initialization to all parameters in the model.

```python
def _init_weights(self, module):
    if isinstance(module, nn.Linear):
        ...
    elif isinstance(module, nn.Embedding):
        ...
```
- **Weight Initialization Logic**: Defines how weights should be initialized for linear and embedding layers, enhancing model training stability and performance.

### Forward Method

```python
def forward(self, idx, targets=None):
```
- **Processing Input**: Takes a batch of token indices (`idx`) and optionally target indices (`targets`).
  - Embeds tokens and adds positional embeddings.
  - Processes the combined embeddings through multiple transformer blocks.
  - Applies final layer normalization and computes logits for each token in the sequence.

```python
if targets is not None:
    ...
```
- **Loss Computation**: If targets are provided, computes the cross-entropy loss between logits and targets, which is used for training.

### Text Generation Method

```python
def generate(self, idx, max_new_tokens):
```
- **Text Generation**: Implements a method to generate text given an initial sequence of token indices (`idx`).
  - Continuously predicts the next token until `max_new_tokens` are generated.
  - Uses the logits to sample the next token probabilistically.
  - Appends the new token to the sequence and repeats.

### Summary
The `GPTLanguageModel` encapsulates the entire functionality required for a GPT-like language model, including token and position encoding, processing through transformer blocks, and generating predictions. It can be used for tasks like text generation, where the model needs to predict the next token in a sequence based on the previous tokens. The model is designed to be flexible and robust, capable of handling sequences up to a defined block size and being trained on a specified vocabulary.

Query: explain to me more clearly what does this do. whats it for and where is it used in the architecture

    def _init_weights(self, module):\
        # Weight Initialization Logic: Defines how weights should be initialized for linear and embedding layers, enhancing model training stability and performance.
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

---

The method `_init_weights` within your `GPTLanguageModel` class is designed to initialize the weights of the neural network components in a specific manner, which is crucial for improving the stability and performance of training deep neural networks like GPT.

### Purpose of Weight Initialization

Proper initialization of model weights can have significant impacts on the training process:
- **Avoids Vanishing/Exploding Gradients:** Good initialization helps in preventing gradients from becoming too small (vanishing) or too large (exploding) during training, which can hinder the learning process.
- **Faster Convergence:** It helps the model to converge faster, reducing the training time.
- **Better Overall Performance:** Proper initialization can lead to better model performance by ensuring that neurons initially activate in a regime of their activation functions that makes learning effective.

### Specifics of the `_init_weights` Method

```python
def _init_weights(self, module):
    # Checks if the module is an instance of nn.Linear
    if isinstance(module, nn.Linear):
        # Initializes weights from a normal distribution with mean 0 and standard deviation 0.02
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        # Initializes biases to zero if the bias is not None (i.e., if biases are used in the layer)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
    # Checks if the module is an instance of nn.Embedding
    elif isinstance(module, nn.Embedding):
        # Initializes embedding weights from a normal distribution with mean 0 and standard deviation 0.02
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
```

### Where It Is Used in the Architecture

- **Application on Modules:** This method is applied to every submodule within your `GPTLanguageModel` through the call `self.apply(self._init_weights)` in the constructor. The `apply` function is a built-in method in `nn.Module` that recursively applies a function to every submodule (including itself).
- **Coverage:** It targets two types of layers—linear layers (`nn.Linear`) and embedding layers (`nn.Embedding`). These layers are fundamental components of the GPT architecture:
  - **Linear Layers:** Used in various places like in the feedforward network (part of each transformer block) and in the output layer to generate logits for each vocabulary token.
  - **Embedding Layers:** Used for converting token indices into embeddings and for adding positional encodings to these embeddings, which are crucial for maintaining the sequence context in models like GPT.

### Summary
The `_init_weights` function is a crucial setup step in preparing the GPT model for training, ensuring that all components start from a consistent, well-considered initial state that promotes effective and stable learning. This step is particularly important in deep learning models, where poor initialization can severely hamper the model's ability to learn.

can you explain to me how does the methods work themselves?


torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
torch.nn.init.zeros_(module.bias)
torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

---

Certainly! The methods you've mentioned are from PyTorch's initialization module (`torch.nn.init`), which provides several functions to set the initial weights and biases of neural network layers. Let's dive into how each of these methods works and why they are used:

### 1. `torch.nn.init.normal_`
```python
torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
```
- **Purpose**: This function initializes the weights of a given tensor (usually a layer's weights) with values drawn from a normal (Gaussian) distribution.
- **Parameters**:
  - `module.weight`: The tensor (weights of a neural network layer) that you want to initialize.
  - `mean=0.0`: The mean (μ) of the normal distribution.
  - `std=0.02`: The standard deviation (σ) of the normal distribution. A standard deviation of 0.02 is often used in transformer models based on empirical results showing it helps in stabilizing training.
- **Operation**: The function directly modifies the tensor `module.weight` in-place by filling it with values sampled from a normal distribution with the specified mean and standard deviation.
- **Usage**: This initialization is typically used for weights to ensure they are neither too small nor too large, avoiding the vanishing or exploding gradients problem in deep networks. It's a common choice for layers in transformer architectures, like GPT.

### 2. `torch.nn.init.zeros_`
```python
torch.nn.init.zeros_(module.bias)
```
- **Purpose**: This function initializes the biases of a given tensor with zero values.
- **Parameters**:
  - `module.bias`: The tensor (biases of a neural network layer) that you want to initialize.
- **Operation**: The function modifies the tensor `module.bias` in-place by setting all its elements to zero.
- **Usage**: Initializing biases to zero is a common practice as it does not add any initial prejudice to the outputs. It ensures that the initial outputs of neurons are solely driven by the weights and not influenced by biases. This can lead to a more stable and predictable start to training.

### 3. Repeated Example
```python
torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
```
- This line is identical to the first example and operates in the same way. It’s repeated usage across different modules in a model helps maintain consistency in how the weights are initialized, supporting overall model stability and convergence during training.

### Summary
These initialization methods are crucial for setting the starting conditions of a neural network:
- **Weights**: Properly initializing weights with `normal_` can greatly influence the learning dynamics, especially important in deep and complex architectures like transformers, where different layers need to start with weights that neither dominate the propagated signals nor vanish them.
- **Biases**: Setting biases to zero with `zeros_` is a safe starting point that contributes to the initial neutrality in decision boundaries, allowing the learning process to evolve naturally based on the data and loss gradients.

By carefully choosing these initialization schemes, you can ensure that your model begins training under conditions that promote effective learning and convergence.

## Back to Coding

In [17]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
                # not just encoding identity of token here. but also its position!
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # initialises 2 embedding layers. REFER TO NOTES ABOVE
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # initialises a sequence of transformer blocks. Which handles the complex interactions between different parts of the input
                                                                                             # through attention mechanisms and feed-forward networks.
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm // applies normalisation to the output of last transformer block
        self.lm_head = nn.Linear(n_embd, vocab_size) # short for language model head // Transforms the output from the dimensionality of embeddings (n_embd) back to the vocabulary size, producing the logits
                                                     # for each token in the vocabulary.

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights) # Customg Weight Initialization: Applies a custom weight initialization to all parameters in the model.

    def _init_weights(self, module):\
        # Weight Initialization Logic: Defines how weights should be initialized for linear and embedding layers, enhancing model training stability and performance.
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) # .normal_ means normal distribution
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias) # initialises the bias to 0 if bias is used.
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape # input dimensions

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
                                                                                # Generates positional embeddings for each position in the sequence up to T. torch.arange(T) creates a tensor from 0 to T-1,
                                                                                # which is then used to fetch positional embeddings. This is crucial as it helps the model understand the order of tokens in the sequence.
        x = tok_emb + pos_emb # (B,T,C) // at this point x not only contains token identity. but also
                                        #  positional identity from position embedding (combines semantic and positional information)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
                         #  Applies a normalization layer (self.ln_f) to the output of the last transformer block, stabilizing the learning process by normalizing the layer outputs.
        logits = self.lm_head(x) # (B,T,vocab_size)
                                 # The final output is transformed by self.lm_head, a linear layer that maps the high-dimensional transformer block outputs to the size of the vocabulary. This produces logits, where each
                                 # element at position [b, t] represents the logit scores (unnormalized probabilities) for predicting each vocabulary token at position t in batch b.
        if targets is None:
            loss = None
        else: # if target is provided. Or loss needs to be calculated
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # reshaping for matmul operations
            targets = targets.view(B*T) # reshaping for matmul operations
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens // crops input context to the maximum context size that the model can handle.
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
                    # torch.multinomial treats each row in probs as a separate distributon and samples num_samples=1 index from each distribution
                    # based on probabilities distribution from softmax // multinomial basically just picks options based on how likely they are to be picked
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [18]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## Notes

The code snippet you provided outlines a typical training loop for a machine learning model using PyTorch. This loop iteratively updates the model's parameters to minimize the loss function over a dataset. Let's break down each part of the loop and explain how it works:

### The Training Loop
```python
for iter in range(max_iters):
```
- **Iteration Control**: The loop iterates over a predefined number of iterations `max_iters`. Each iteration represents a single step of training where the model parameters are updated.

### Periodic Evaluation
```python
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
```
- **Conditional Evaluation**: At regular intervals (`eval_interval`), and at the last iteration, the model's performance is evaluated on both the training and validation datasets.
- **Loss Calculation**: The `estimate_loss()` function is called, which likely computes the average loss over several batches of both training and validation data without affecting the model’s parameters (as it should be run in evaluation mode).
- **Logging**: The losses are printed out, giving a snapshot of the model's performance at that point in training. This helps in monitoring the training progress and debugging if necessary.

### Sampling a Batch of Data
```python
    xb, yb = get_batch('train')
```
- **Data Fetching**: A batch of data (`xb` for inputs, `yb` for targets) is fetched from the training dataset using the `get_batch` function. This function probably handles the data loading and batching operations, ensuring that the model sees different parts of the dataset in each epoch.

### Loss Calculation
```python
    logits, loss = model(xb, yb)
```
- **Forward Pass**: The model processes the batch of inputs (`xb`) to produce outputs (`logits`) and computes the loss (`loss`) by comparing the outputs to the actual targets (`yb`).

### Gradient Computation and Parameter Update
```python
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
```
- **Zeroing Gradients**: Before computing the gradients, the gradients of all model parameters are set to zero (or `None`, which can be slightly more efficient). This is necessary because gradients are accumulated by default, for cases involving RNNs or when multiple sub-parts of the dataset are processed before a step of the optimizer.
- **Backward Pass**: The `loss.backward()` computes the gradients of the loss with respect to the model parameters. This is where PyTorch's autograd system comes into play, calculating gradients efficiently through the computation graph.
- **Parameter Update**: `optimizer.step()` updates the model parameters using the computed gradients. The type of update depends on the optimizer used (e.g., SGD, Adam).

### Summary
This training loop is fundamental for training deep learning models. It repeatedly adjusts the model's weights to minimize the loss on a training set, evaluates performance on a validation set to monitor overfitting, and provides feedback on the training progress. Properly managing these steps ensures effective training and is crucial for developing accurate models.

## back to coding

In [20]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2221, val loss 4.2306
step 500: train loss 1.7526, val loss 1.9053
step 1000: train loss 1.3949, val loss 1.6050
step 1500: train loss 1.2625, val loss 1.5219
step 2000: train loss 1.1860, val loss 1.5046
step 2500: train loss 1.1254, val loss 1.4972
step 3000: train loss 1.0694, val loss 1.4849
step 3500: train loss 1.0211, val loss 1.5048
step 4000: train loss 0.9643, val loss 1.5160
step 4500: train loss 0.9121, val loss 1.5396
step 5000: train loss 0.8673, val loss 1.5552
step 5500: train loss 0.8052, val loss 1.5988
step 6000: train loss 0.7611, val loss 1.6231
step 6500: train loss 0.7087, val loss 1.6706
step 7000: train loss 0.6644, val loss 1.7000
step 7500: train loss 0.6187, val loss 1.7484
step 8000: train loss 0.5818, val loss 1.7882
step 8500: train loss 0.5350, val loss 1.8304
step 9000: train loss 0.4973, val loss 1.8688
step 9500: train loss 0.4638, val loss 1.9050
step 9999: train loss 0.4333, val loss 1.9475


In [21]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device) # basically inputtng nothing as context for model
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


KING RICHARD III:
Mistakest me haste; become enough that I was.
She what hast thou torments now, thou rose
And here should not shame thee says a strength shore,
To thiry by his country of smiles' tongue
And seem chase them ere in parchasing toward the shards,
As doth aid office and I am corn
Till it from the end hath my hazard.
He hath at said throws for the time butcherly,
That he had nothing but wrong'd, made hauntings
And all these eyes did eal his despitions,
Kill dis the enemies regain a cr


In [22]:
torch.save(model, 'full_model.pth')